In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load the CSV file containing the User IDs and URLs
df_csv = pd.read_csv('User_Review_url_and_ID.csv')

# Headers to simulate a browser
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "max-age=0",
    "cookie": 'ccsid=997-0059822-9974992; locale=en; blocking_sign_in_interstitial=true; csm-sid=448-9044706-8238887; session-id=138-9449735-4650719; lc-main=en_US; ubid-main=132-0831767-7645715; csm-hit=tb:s-HH6TEBC541XWDPX2HW08|1728400665806&t:1728400665807; session-id-time=2359120669l; session-token="9epd7M6HsVRPHteGVvJu5Fk8d+KvASerta8tz+la5w+TVREtncxVnfcCzs8K9EpSg+fQrgx1GgnsD2rLlayR4DmjBe1MsI/hzqbpLxW9fzltLac/Bx4GP/iysuf1f336QEHPYkFM5wvwKAJRlJF/gUtvoqUNgeHO7d0nUgwzQ4n10cVaWcJP8kU3S8U/EG5f1z4kuWhJEC/GxOGAFPM2AwR1aAfDTGc3oUmUkI4heyVFL2PVr5DhINiO23tre2UqOAtOSgArJ6foDhvdkRwB1A2cwygZeAAPu6znpJ+cxdwCK+Qw0zKdF5L4dt5MPkikWWxVqpvSWD4krg+mGhL2EJhFdAQbaWn2SM7qiWrwNN8="; x-main="qY09NsWCzSNW8b933Yonj4af6qs5TDB0DUsYapgS18@nyKFkHdJj5p@Mt2dldE6F"; at-main=Atza|IwEBIKKe6dZi9mkGtv2CtS3qsv9NuaJNSNpmNpDSxRXvt17a_x2-zjEpiAOHkEQ34JuOVngoGfAl8dIqliYqOigT30w9CGWKtZI6rgDWz0mruXs-yNgr_nsxBWgaXid-1e8ZvmgQryKo5OgAIx-Lce0ocNWWSfh3g77KeZNETFdonWPAuJz5V82UuuKblmr-o03ZzeyBC7rbWUbFLRhtcSWy7Mqogs8ONE5dM3XgdBITj5b5F2h4Z_ih2WgCM4wTps3r6HY; sess-at-main="l6swvzfG2thASaG98K8Nu/j0D/+teOfY5a1UhpsZBz8="; likely_has_account=true; *session*id2=4cd4cabfc8c16e891c0f62348aa076dd',
    "if-none-match": 'W/"862d60fdc6915eed33d69a373690f36d"',
    "priority": "u=0, i",
    "sec-ch-ua": '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

# Function to scrape a single user's review page
def scrape_user_reviews(user_id, page_num):
    url = f"https://www.goodreads.com/review/list/{user_id}?order=d&page={page_num}&sort=rating&view=reviews"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    ratings_data = []
    rows = soup.find_all('tr', class_='bookalike review')

    for row in rows:
        book_td = row.find('td', class_='field title')
        if book_td:
            bb = book_td.find('div', class_='value')
            a_tag = bb.find('a', href=True)
            if a_tag and '/book/show/' in a_tag['href']:
                book_url = a_tag['href']
                book_id = book_url.split('/book/show/')[1].split('-')[0].split('.')[0]
                rating_td = row.find('td', class_='field rating')
                if rating_td:
                    stars = rating_td.find_all('span', class_='staticStar p10')
                    rating = len(stars)
                    ratings_data.append({
                        'User_ID': user_id,
                        'Book_ID': book_id,
                        'Rating': rating
                    })
    return ratings_data

# Multithreaded execution
def run_scraping():
    ratings_data = []

    with ThreadPoolExecutor(max_workers=10) as executor:  # You can adjust max_workers
        futures = []
        for index, row in df_csv.iterrows():
            user_id = row['User_ID']
            for page_num in range(1, 4):  # Adjust the range as necessary
                futures.append(executor.submit(scrape_user_reviews, user_id, page_num))

        for future in as_completed(futures):
            result = future.result()
            if result:
                ratings_data.extend(result)

    return ratings_data

# Get the scraped data
ratings_data = run_scraping()

# Convert to DataFrame
ratings_df = pd.DataFrame(ratings_data)

# Optionally, save to a CSV file
ratings_df.to_csv('scraped_ratings1.csv', index=False)
